# Light Beads Microscopy (LBM) Processing Pipeline

A python implementation of an analytical pipeline for LBM data. This makes use of the following python libraries (with their own dependencies):

- [suite2p](https://www.biorxiv.org/content/10.1101/061507v2.full.pdf)
- [CaImAn](https://github.com/flatironinstitute/CaImAn/tree/main)

Some challanges I see considering the nature of the data:
- Scalability: These are very large file sizes, memory usage and performance are both a concern


#### Recording parameters:
- 2 x 2 mm FOV
- 6.45 Hz volume rate
- 30 planes
- 75% power, 250 mW


Several libraries can be used to load `.tif` files.
I will use [ScanImageTiffReader](https://vidriotech.gitlab.io/scanimagetiffreader-python/) because it is the utility chosen by [suite2p](https://github.com/MouseLand/suite2p).
However, this is mesoscopic data with multiple axial planes, which the current python ecosystem does not handle well.
We can handle that.

In [8]:
from pathlib import Path  # useful for cross-platform compatibility and path manipulationimport
import ScanImageTiffReader
import re
import json

path = Path(r"C:\Users\Flynn\Dropbox\RBO\LBM-sample\spon_6p45Hz_2mm_75pct_00001_00001.tif")
reader = ScanImageTiffReader.ScanImageTiffReader(str(path))
metadata_str = reader.metadata()
data = reader.data()

## Metadata Parsing

You'll notice the metadata is a string separated by newline chars. We can't deserialize the json portion until all of the `SI` metadata groups are separated, so we have a valid json string.

In [9]:
def parse_value(value_str):
    """
    Parse the output of the ScanImageTiffReader metadata parser.
    Despite the documentation, the parser does not return a valid json string.
    The function handles various types like strings, numbers, arrays, and special values.

    Args:
    value_str (str): The string representation of the value.

    Returns:
    Union[str, int, float, list, bool]: The parsed value in its appropriate data type.

    Examples:
    >>> parse_value("'Hello'")
    'Hello'
    >>> parse_value("5.6")
    5.6
    >>> parse_value("[1 2 3]")
    [1, 2, 3]
    >>> parse_value("NaN")
    nan
    """
    if value_str.startswith("'") and value_str.endswith("'"):
        return value_str[1:-1]
    if value_str == 'true':
        return True
    if value_str == 'false':
        return False
    if value_str == 'NaN':
        return float('nan')
    if value_str == 'Inf':
        return float('inf')
    if re.match(r'^\d+(\.\d+)?$', value_str):
        return float(value_str) if '.' in value_str else int(value_str)
    if re.match(r'^\[(.*)\]$', value_str):
        return [parse_value(v.strip()) for v in value_str[1:-1].split()]
    return value_str


def parse_key_value(parse_line):
    """
    Parse a line containing a key-value assignment to extract the key and its associated value.

    Args:
    parse_line (str): The string containing the key-value assignment.

    Returns:
    Tuple[str, Union[str, int, float, list, bool]]: A tuple containing the key and the parsed value.

    Examples:
    >>> parse_key_value("SI.VERSION = '2018b'")
    ('SI.VERSION', '2018b')
    >>> parse_key_value("SI.VALUE = 5.6")
    ('SI.VALUE', 5.6)
    >>> parse_key_value("SI.ARRAY = [1 2 3]")
    ('SI.ARRAY', [1, 2, 3])
    """
    key_str, value_str = parse_line.split(' = ', 1)
    return key_str, parse_value(value_str)


lines = metadata_str.split('\n')
metadata_dict = {}
json_portion = []
parsing_json = False
for line in lines:
    line = line.strip()
    if not line:
        continue
    if line.startswith('SI.'):
        key, value = parse_key_value(line)
        metadata_dict[key] = value
    elif line.startswith('{'):
        parsing_json = True
    if parsing_json:
        json_portion.append(line)

metadata_json = json.loads('\n'.join(json_portion))

#### Our data is three-dimensional: [ntz, ny, nx]
Where:
1) `ntz` (Time x Plane) Dimension:
    - `nt` is the number of time points.
    - `nz` is the number of planes.
    - The ntz dimension spans through the time points sequentially for each plane.
    - For `t=0`, you have data for all planes from `z=0` to `z=29`, same yields true for `t=1, t=2...t=n`.
2) `ny` (Vertical) and `nx` (Horizontal) Dimensions:
    - This represents the Field of View (FOV) in the corresponding direction.
    - Given a 2 x 2 mm FOV, then each pixel in the ny dimension would correspond to 2 mm / `[ny or ny].size`.
    - However, each `ny` is the vertical concatenation of all strips in the lateral FOV

In [ ]:
# Extract the nz/nt given the number of planes
num_planes = 30
num_timepoints = data.shape[0] // num_planes
reshaped_data = data.reshape(num_timepoints, num_planes, *data.shape[1:])
# The last plane can be discarded
valid_data = reshaped_data[:, :num_planes - 1, :, :]

New dimensions are now `[nt, nz, ny, nx]`
..or [time, planes, height, width]
The temporal interval between each time point can be determined using the volume rate (6.45 Hz).
Each time point is separated by approximately 1/6.45 seconds.

In [10]:
vol = metadata_dict["SI.hRoiManager.scanVolumeRate"]